In [1]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install imutils

  Using cached imutils-0.5.3.tar.gz (17 kB)
  Created wheel for imutils: filename=imutils-0.5.3-py3-none-any.whl size=25856 sha256=69d6d6d89b76d0383a0cf5af64190a39a1109beab42248a7dfbfc1ca6e8c61fc
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\27\4d\3a\6dcdf7c3ebc87bf1ae013d96c9cf060ccfe334bb5ee769f377
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


In [11]:
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread
import numpy as np
#import argparse
import imutils
import time
import dlib
import cv2
import os

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    C = dist.euclidean(eye[0], eye[3])

    ear = (A + B) / (2.0 * C)

    return ear

def final_ear(shape):
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]

    leftEAR = eye_aspect_ratio(leftEye)
    rightEAR = eye_aspect_ratio(rightEye)

    ear = (leftEAR + rightEAR) / 2.0
    return (ear, leftEye, rightEye)  #Here we are returning lefteye and righteye coordinates as well as we are displaying these pts in the video

def lip_distance(shape):
    top_lip = shape[50:53]
    top_lip = np.concatenate((top_lip, shape[61:64]))

    low_lip = shape[56:59]
    low_lip = np.concatenate((low_lip, shape[65:68]))

    top_mean = np.mean(top_lip, axis=0)
    low_mean = np.mean(low_lip, axis=0)

    distance = abs(top_mean[1] - low_mean[1])
    return distance


EYE_AR_THRESH = 0.3   #threshold for eyes
EYE_AR_CONSEC_FRAMES = 30   #30 is the frame number ie we check for 30 frames and then declare the user is sleeping or not
YAWN_THRESH = 20   #threshold for yawning
alarm_status = False
alarm_status2 = False
saying = False
COUNTER = 0

#print("-> Loading the predictor and detector...")
detector = dlib.get_frontal_face_detector()  #detects the face from the frame
predictor = dlib.shape_predictor(r'C:\Users\DELL\Desktop\shape_predictor_68_face_landmarks.dat')


cap = cv2.VideoCapture(0)

time.sleep(1.0)

while True:

    _, frame = cap.read()
    #frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    rects = detector(gray)
    

    for rect in rects:
        
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        eye = final_ear(shape)
        ear = eye[0]
        leftEye = eye [1]
        rightEye = eye[2]

        distance = lip_distance(shape)

        #to plot the ouline of the points in the video
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

        lip = shape[48:60]
        cv2.drawContours(frame, [lip], -1, (0, 255, 0), 1)

        if ear < EYE_AR_THRESH:    #comparing value of ear with the threshold value
            COUNTER += 1

            if COUNTER >= EYE_AR_CONSEC_FRAMES:   #if the counter value is greater than number of frames then the persion is drowsy

                cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        else:
            COUNTER = 0

        if (distance > YAWN_THRESH):
                cv2.putText(frame, "Yawn Alert", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        

        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "YAWN: {:.2f}".format(distance), (300, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)


    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
